<a href="https://colab.research.google.com/github/abir101-prog/pytorch_experiments/blob/main/pytorch_tutorial_example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
torch.cuda.is_available()   # cuda not available: sad

False

## Getting started

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



#### Dataloader wraps an iterable over dataset. as well as automatic batching, sampling, shuffling, multiprocess data loading

In [ ]:
batch_size = 64

# create dataloader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f'Shape of X [N C H W]: {X.shape}')
  print(f'Shape of y: {y.shape} {y.dtype}')
  break

Shape of X [N C H W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Model

neural net is a class inherited from ```nn.Module```. 


```__init__()```  contains layers, 
forward() defines how data is to be passed through the network
```forward()``` defines how data is to be passed through the network

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# MODEL
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### optimizing Model parameters

In [ ]:
# lossfuntion:
loss_fn = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute prediction error
    pred = model(X);
    loss = loss_fn(pred, y)

    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n--------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("bruh")

Epoch 1
--------------------------
loss: 2.287969 [    0/60000]
loss: 2.283871 [ 6400/60000]
loss: 2.272103 [12800/60000]
loss: 2.277551 [19200/60000]
loss: 2.247924 [25600/60000]
loss: 2.221306 [32000/60000]
loss: 2.212195 [38400/60000]
loss: 2.176763 [44800/60000]
loss: 2.174982 [51200/60000]
loss: 2.153114 [57600/60000]
Test Error: 
 Accuracy: 40.4%, Avg loss: 2.143618 

Epoch 2
--------------------------
loss: 2.145219 [    0/60000]
loss: 2.141461 [ 6400/60000]
loss: 2.084281 [12800/60000]
loss: 2.111569 [19200/60000]
loss: 2.058069 [25600/60000]
loss: 1.989487 [32000/60000]
loss: 2.016697 [38400/60000]
loss: 1.933525 [44800/60000]
loss: 1.943694 [51200/60000]
loss: 1.874964 [57600/60000]
Test Error: 
 Accuracy: 58.7%, Avg loss: 1.873318 

Epoch 3
--------------------------
loss: 1.903277 [    0/60000]
loss: 1.872417 [ 6400/60000]
loss: 1.756305 [12800/60000]
loss: 1.806212 [19200/60000]
loss: 1.700454 [25600/60000]
loss: 1.645794 [32000/60000]
loss: 1.669124 [38400/60000]
loss: 1.

### Saving the trained model:

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved pytorch model state to model.pth")

Saved pytorch model state to model.pth


### Reloading the saved model

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
